In [48]:
import glob
import logging
import os
import pickle
import random
import re
import shutil
import copy
from typing import Dict, List, Tuple
import nltk
import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange
from transformers import (MODEL_WITH_LM_HEAD_MAPPING, WEIGHTS_NAME, AdamW, AutoConfig, AutoModelWithLMHead, AutoTokenizer, 
                          PreTrainedModel, PreTrainedTokenizer, get_linear_schedule_with_warmup, BertModel, DistilBertModel, AlbertModel)

try:
    from torch.utils.tensorbord import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

In [90]:
class Args():
    
    model_type='dbert'
    model_name_or_path="distilbert-base-uncased"
    cache_dir=None
    run_id=1
    device=-1
    batch_size=32
    
    pickle_file_train='gender_stereo_data_for_inlp_train'
    pickle_file_dev='gender_stereo_data_for_inlp_dev'
    
args=Args()

In [91]:
def load_pickle(pickle_file):
    with open("./data_for_inlp/{}/{}.pickle".format(args.model_type, pickle_file), "rb") as f:
        data=pickle.load(f)
        txts=[d["text"] for d in data]
    return txts

In [92]:
device="cpu" if args.device==-1 else "cuda:{}".format(args.device)

config=AutoConfig.from_pretrained(args.model_name_or_path, cache_dir=args.cache_dir)
config.output_hidden_states='true'

pretrained_weights=args.model_name_or_path

if args.model_type=='bert':
    model=BertModel.from_pretrained(pretrained_weights, config=config, cache_dir=args.cache_dir)
elif args.model_type=='dbert':
    model=DistilBertModel.from_pretrained(pretrained_weights, config=config, cache_dir=args.cache_dir)

tokenizer=AutoTokenizer.from_pretrained(args.model_name_or_path, cache_dir=args.cache_dir)

block_size=tokenizer.model_max_length

#model=BertModel.from_pretrained(args.model_name_or_path, from_tf=bool(".ckpt" in args.model_name_or_path), config=config, cache_dir=args.cache_dir)

model.to(device)
model.eval()

rand_seed=args.run_id

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [93]:
if not os.path.exists("encodings"):
    os.makedirs("encodings")

In [94]:
data_file_train=load_pickle(args.pickle_file_train)
data_file_dev=load_pickle(args.pickle_file_dev)

In [95]:
def encode(nlp_model, nlp_tokenizer, texts, args):
    
    all_H=[]
    nlp_model.eval()
    with torch.no_grad():
        
        print("Encoding...")
        batch_size=args.batch_size
        pbar=tqdm(range(len(texts)), ascii=True)
        
        for i in range(0, len(texts)-batch_size, batch_size):
            
            batch_texts=texts[i: i+batch_size]
            
            batch_encoding=nlp_tokenizer.batch_encode_plus(batch_texts, padding=True, max_length=512, truncation=True)

            if args.model_type=='bert':
                input_ids, token_type_ids, attention_mask=batch_encoding["input_ids"], batch_encoding["token_type_ids"], batch_encoding["attention_mask"]
                input_ids=torch.tensor(input_ids).to(device)
                token_type_ids=torch.tensor(token_type_ids).to(device)
                attention_mask=torch.tensor(attention_mask).to(device)
                H=nlp_model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).pooler_output
            elif args.model_type=='dbert':
                input_ids, attention_mask=batch_encoding["input_ids"], batch_encoding["attention_mask"]
                input_ids=torch.tensor(input_ids).to(device)
                attention_mask=torch.tensor(attention_mask).to(device)
                #below is the equivalent to the pooler_ouput from BERT for the DistilBert model; a fixed-size vector representation of the entire input sequence
                hiddens=nlp_model(input_ids=input_ids, attention_mask=attention_mask)[0]
                H=hiddens[:, 0, :]
            
            assert len(H.shape)==2
            all_H.append(H.detach().cpu().numpy())
            
            pbar.update(batch_size)
        
        remaining=texts[(len(texts)//args.batch_size)*args.batch_size:]
        if len(remaining)>0:
            
            batch_encoding=nlp_tokenizer.batch_encode_plus(remaining, padding=True, max_length=512, truncation=True)

            if args.model_type=='bert':
                input_ids, token_type_ids, attention_mask=batch_encoding["input_ids"], batch_encoding["token_type_ids"], batch_encoding["attention_mask"]
                input_ids=torch.tensor(input_ids).to(device)
                token_type_ids=torch.tensor(token_type_ids).to(device)
                attention_mask=torch.tensor(attention_mask).to(device)
                H=nlp_model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).pooler_output
            elif args.model_type=='dbert':
                input_ids, attention_mask=batch_encoding["input_ids"], batch_encoding["attention_mask"]
                input_ids=torch.tensor(input_ids).to(device)
                attention_mask=torch.tensor(attention_mask).to(device)
                #below is the equivalent to the pooler_ouput from BERT for the DistilBert model; a fixed-size vector representation of the entire input sequence
                hiddens=nlp_model(input_ids=input_ids, attention_mask=attention_mask)[0]
                H=hiddens[:, 0, :]

            #H=nlp_model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).pooler_output
            
            assert len(H.shape)==2
            all_H.append(H.detach().cpu().numpy())
        
    H_np=np.concatenate(all_H)
    assert len(H_np.shape)==2
    assert len(H_np)==len(texts)
    return H_np

In [96]:
H_train=encode(model, tokenizer, data_file_train, args)

Encoding...




  0%|          | 0/8977 [00:00<?, ?it/s]

  0%|          | 32/8977 [00:01<09:05, 16.39it/s]

  1%|          | 64/8977 [00:03<07:34, 19.62it/s]

  1%|1         | 96/8977 [00:04<06:14, 23.70it/s]

  1%|1         | 128/8977 [00:06<06:56, 21.22it/s]

  2%|1         | 160/8977 [00:07<06:46, 21.67it/s]

  2%|2         | 192/8977 [00:08<06:21, 23.04it/s]

  2%|2         | 224/8977 [00:09<05:57, 24.47it/s]

  3%|2         | 256/8977 [00:10<05:22, 27.01it/s]

  3%|3         | 288/8977 [00:11<05:03, 28.64it/s]

  4%|3         | 320/8977 [00:13<05:23, 26.74it/s]

  4%|3         | 352/8977 [00:14<05:34, 25.77it/s]

  4%|4         | 384/8977 [00:15<05:20, 26.85it/s]

  5%|4         | 416/8977 [00:16<05:21, 26.67it/s]

  5%|4         | 448/8977 [00:18<06:01, 23.59it/s]

  5%|5         | 480/8977 [00:19<05:49, 24.33it/s]

  6%|5         | 512/8977 [00:20<05:20, 26.42it/s]

  6%|6         | 544/8977 [00:21<04:45, 29.58it/s]

  6%|6         | 576/8977 [00:22<04:42, 29.74it/s]

  7%|6         | 608/89

In [97]:
H_dev=encode(model, tokenizer, data_file_dev, args)

Encoding...




  0%|          | 0/997 [00:00<?, ?it/s]

  3%|3         | 32/997 [00:01<00:36, 26.69it/s]

  6%|6         | 64/997 [00:02<00:38, 24.40it/s]

 10%|9         | 96/997 [00:03<00:34, 25.80it/s]

 13%|#2        | 128/997 [00:05<00:34, 24.86it/s]

 16%|#6        | 160/997 [00:06<00:34, 24.37it/s]

 19%|#9        | 192/997 [00:07<00:30, 26.17it/s]

 22%|##2       | 224/997 [00:09<00:33, 23.18it/s]

 26%|##5       | 256/997 [00:11<00:36, 20.29it/s]

 29%|##8       | 288/997 [00:13<00:37, 18.99it/s]

 32%|###2      | 320/997 [00:14<00:32, 20.70it/s]

 35%|###5      | 352/997 [00:15<00:31, 20.70it/s]

 39%|###8      | 384/997 [00:17<00:27, 22.27it/s]

 42%|####1     | 416/997 [00:18<00:23, 24.35it/s]

 45%|####4     | 448/997 [00:19<00:21, 25.46it/s]

 48%|####8     | 480/997 [00:20<00:18, 27.57it/s]

 51%|#####1    | 512/997 [00:21<00:16, 28.89it/s]

 55%|#####4    | 544/997 [00:22<00:15, 29.25it/s]

 58%|#####7    | 576/997 [00:23<00:13, 30.84it/s]

 61%|######    | 608/997 [00:24<00:14, 27.

In [98]:
if not os.path.exists("encodings/{}".format(args.model_type)):
    os.makedirs("encodings/{}".format(args.model_type))

path_train="encodings/{}/{}.npy".format(args.model_type, args.pickle_file_train)
np.save(path_train, H_train)

path_dev="encodings/{}/{}.npy".format(args.model_type, args.pickle_file_dev)
np.save(path_dev, H_dev)